In [ ]:
import pandas as pd

# Get table from website
df = pd.read_html("https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union-0")[0]
df.columns = df.iloc[0]
df = df[1:]

# Drop rows not describing a SOTU
df = df.drop(df[df.President.str.contains("NOT a ",na=False)].index)

# Forward fill president names
df.fillna(method="ffill", inplace=True)

# Remove the rows that contain aggregated data
df = df.drop(df[df.Date.str.contains("average")].index)

# Only keep spoken speeches
df = df[df.Format == 'spoken']

# Replace approximations by the actual numbers
df.Minutes.replace(r"apx. ([0-9]+) min.", r"0:\1:00", regex=True, inplace=True)

# Convert to seconds
df["Seconds"] = pd.to_timedelta(df.Minutes).dt.total_seconds()

# Remove unused columns
df.drop(columns=['Minutes', 'Format'], inplace=True)

# Extract the year and make it the index
df[['Month', 'Year']] = df.Date.str.split(',', expand=True)
df = df.drop(columns=['Date', 'Month']).set_index('Year')
df

In [ ]:
# Write to file
df.to_csv('speeches-meta.csv')